<a href="https://colab.research.google.com/github/Shrivastav-Gaurav/GenAI-ML-Notebook/blob/main/RAG_application.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# downloading packages for running the notebook
import sys
import subprocess

# subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-r', 'requirements.txt', '--quiet'])torch==2.1.1
!pip install gradio==3.38.0
!pip openai==0.27.8
!pip fasttext==0.9.2
!pip faiss-cpu==1.7.4
!pip torchaudio==2.1.1
!pip langchain==0.0.243
!pip llama-index==0.6.8
!pip torchvision==0.16.1
!pip transformers==4.31.0
!pip gradio_client==0.2.10
!pip install sentence-transformers
!pip install langchain llama-index
!pip install fastText
!pip install langchain-community langchain-core
!pip install langchain_huggingface
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 761.8 kB/s eta 0:00:00
ERROR: Operation cancelled by user
ERROR: unknown command "openai==0.27.8"
ERROR: unknown command "fasttext==0.9.2"
ERROR: unknown command "faiss-cpu==1.7.4"
ERROR: unknown command "torchaudio==2.1.1"
ERROR: unknown command "langchain==0.0.243"
Traceback (most recent call last):
  File "<frozen importlib._bootstrap>", line 688, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 883, in exec_module
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/spinners.py", line 9, in <module>
    from pip._internal.utils.logging import get_indentation
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/utils/logging.py", line 13, in <module>
    from pip._vendor.rich.console import (
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/rich/console.py", line 60, in <module>
    from .pre

In [ ]:
import os
import openai
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
import getpass

In [ ]:
os.environ['OPENAI_API_KEY'] = getpass.getpass()

··········


# Language Detection Model from NLLB

It adds robustness in the approach to handle multilingual inputs and translate them if required.



In [ ]:
import fasttext
from huggingface_hub import hf_hub_download

model_path = hf_hub_download(repo_id="facebook/fasttext-language-identification", filename="model.bin")
model = fasttext.load_model(model_path)

model.bin:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

# Text Cleaning and Preprocessing

In [ ]:
with open('dataset.txt', 'r') as f:
    data = f.read()

data = data.replace('\n\n','\n')
data = data.split('---')

In [ ]:
for i in range(0, len(data)):
    if i==4:
      data[i] = data[i].replace('\n**', '\n###').replace('**','')
    elif i==3:
      data[i] = data[i].replace('**','')
    else:
      data[i] = data[i].replace('**','')

In [ ]:
ques_ans = dict()
for i in range(0, len(data)):
    topics = data[i].split('\n###')
    for topic in topics[1:]:
      question_answer_pair = topic.split('\n')
      ques_ans[question_answer_pair[0]] = " ".join(question_answer_pair[1:])

In [ ]:
all_content = str()
for key, value in ques_ans.items():
    # print(key)
    all_content += key + " " + value + "\n"
print(all_content)

 What is Pan card?
 Who needs a Pan card?
 Types of PAN cards
 Why do NRIs need PAN card?
 Importance of PAN card for different NRI account options
 How can NRI apply for a new PAN card
 Documents required for a new PAN Card
 Cost of new PAN card
 Time required to issue PAN card
 Information that can be updated in the PAN Card
 General process to update details on PAN Card
 Documents required to update the details on PAN Card
 Documents required to update the address on PAN Card
 Process to change the citizenship on PAN Card
 Reprinting lost Aadhaar Card
 Documents required for reprinting the lost PAN card
 Charges for reprinting the PAN Card
 Linking PAN with Aadhaar card
 Process to link PAN with Aadhaar
 ABC fees to link PAN with Aadhaar
 Documents required to link PAN with Aadhaar
 Time required for PAN Aadhaar link for NRI?
 What is Form 49aa?
 Documents Required for Form 49AA
Is it mandatory to link Aadhaar with PAN for NRI?
Is PAN card different for NRI?
Can I apply for pan card

## Text Chunking

In [ ]:
text_splitter = CharacterTextSplitter(separator='\n', chunk_size=300, chunk_overlap=128, length_function=len)

chunks = text_splitter.split_text(all_content)

In [ ]:
from sentence_transformers import SentenceTransformer
text = "Sampel text!"
encoder = SentenceTransformer("paraphrase-mpnet-base-v2")
vectors = encoder.encode(text)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
embeddings = OpenAIEmbeddings()

vectorStore = FAISS.from_texts(chunks, embeddings)
vectorStore.save_local("faiss_doc_idx")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [ ]:
docs = vectorStore.similarity_search("WHow lowng does it usually take to receive the PAN card after applying?")

In [ ]:
from langchain_community.llms import HuggingFaceHub

In [ ]:
import os

In [ ]:
from langchain.llms import OpenAI
from langchain.llms import huggingface_hub
from langchain.callbacks import get_openai_callback
from langchain.chains.question_answering import load_qa_chain

llm = OpenAI(temperature=0)

chain = load_qa_chain(llm, chain_type="refine")

with get_openai_callback() as cb:
    response = chain.run(input_documents=[docs[0]], question=chain)

In [ ]:
response

'\nThe time required to issue a PAN card depends on whether you have an Aadhaar card or not. If you have an Aadhaar card, you can get a PAN card instantly (in under 10 minutes) by applying through ABC. However, if you do not have an Aadhaar card, the process will take 3 weeks after the payment is made to ABC. Additionally, if you need to update or correct any information on your PAN card, you can do so by providing the necessary documents and information to ABC.'

# Gradio Chatbot

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage
import openai
import gradio as gr

def predict(message, history):
    history_langchain_format = []

    for human, ai in history:
        history_langchain_format.append(HumanMessage(content=human))
        history_langchain_format.append(AIMessage(content=ai))

    #history_langchain_format.append(HumanMessage(content=message))

    language = model.predict(message)[0][0].split('__')[-1]
    template = """I want you to act as a question answering bot which uses the context mentioned and answer in a concise manner and doesn't make stuff up.
            You will answer question based on the context - {context}.
            You will create content in""" + str(language) + """language.
            Question: {question}
            Answer:
            """
    QA_CHAIN_PROMPT = PromptTemplate.from_template(template)
    qa_chain = RetrievalQA.from_chain_type(llm,retriever=vectorStore.as_retriever(), chain_type_kwargs={"prompt": QA_CHAIN_PROMPT})

    result = qa_chain({"query": message})

    history_langchain_format.append(HumanMessage(content=message))
    history_langchain_format.append(AIMessage(content=result['result']))

    return result['result']

gr.ChatInterface(predict,
    chatbot=gr.Chatbot(height=300),
    textbox=gr.Textbox(placeholder="Ask me a question related to PAN Services", container=False, scale=7),
    title="DocumentQABot",
    theme="soft",
    examples=["What is the cost/fees of a PAN card?", "How long does it usually take to receive the PAN card after applying?"],
    retry_btn=None,
    undo_btn="Delete Previous",
    clear_btn="Clear",).launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
IMPORTANT: You are using gradio version 3.38.0, however version 4.29.0 is available, please upgrade.
--------
Running on public URL: https://64ff6dc2a444cba4ea.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
